In [1]:
import pandas as pd

df = pd.read_csv('telco-churn-customer.csv') 
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [2]:
df.info()
df.isnull().sum()
df['TotalCharges'].unique()[:10]  # Preview weird values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


array(['29.85', '1889.5', '108.15', '1840.75', '151.65', '820.5',
       '1949.4', '301.9', '3046.05', '3487.95'], dtype=object)

In [3]:
import pandas as pd

# Step 1: Load again just in case
df = pd.read_csv("telco-churn-customer.csv")

# Step 2: Strip whitespace from all string entries
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Step 3: Convert 'TotalCharges' to numeric, forcing errors to NaN
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Step 4: Drop rows where TotalCharges couldn't convert
df = df.dropna(subset=['TotalCharges'])

# Step 5: Final conversion to float (optional, usually already float by now)
df['TotalCharges'] = df['TotalCharges'].astype(float)

# Step 6: Confirm it's clean
print(df['TotalCharges'].head())
print(df.info())


0      29.85
1    1889.50
2     108.15
3    1840.75
4     151.65
Name: TotalCharges, dtype: float64
<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7032 non-null   object 
 1   gender            7032 non-null   object 
 2   SeniorCitizen     7032 non-null   int64  
 3   Partner           7032 non-null   object 
 4   Dependents        7032 non-null   object 
 5   tenure            7032 non-null   int64  
 6   PhoneService      7032 non-null   object 
 7   MultipleLines     7032 non-null   object 
 8   InternetService   7032 non-null   object 
 9   OnlineSecurity    7032 non-null   object 
 10  OnlineBackup      7032 non-null   object 
 11  DeviceProtection  7032 non-null   object 
 12  TechSupport       7032 non-null   object 
 13  StreamingTV       7032 non-null   object 
 14  StreamingMovies   7032 non-null   object 

C:\Users\User\AppData\Local\Temp\ipykernel_16496\2775822594.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [4]:
# Average customer lifetime value
df['CLTV'] = df['MonthlyCharges'] * df['tenure']
df[['customerID', 'MonthlyCharges', 'tenure', 'CLTV']].head()

,customerID,MonthlyCharges,tenure,CLTV
0,7590-VHVEG,29.85,1,29.85
1,5575-GNVDE,56.95,34,1936.30
2,3668-QPYBK,53.85,2,107.70
3,7795-CFOCW,42.30,45,1903.50
4,9237-HQITU,70.70,2,141.40


In [5]:
df.to_csv("telco_cleaned.csv", index=False)

In [6]:
import pandas as pd
import sqlite3

# Load your cleaned CSV (optional if still in memory)
df = pd.read_csv("telco_cleaned.csv")

# Connect to SQLite (creates telco.db file if it doesn't exist)
conn = sqlite3.connect("telco.db")


In [7]:
# Create 'customers' table (replace if exists)
df.to_sql("customers", conn, if_exists='replace', index=False)


7032

In [8]:
# Example: churn rate by contract type
query = """
SELECT Contract, COUNT(*) AS Total,
       SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
       ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM customers
GROUP BY Contract
ORDER BY ChurnRate DESC;
"""

result = pd.read_sql_query(query, conn)
print(result)

         Contract  Total  Churned  ChurnRate
0  Month-to-month   3875     1655      42.71
1        One year   1472      166      11.28
2        Two year   1685       48       2.85


SECTION A: CHURN INSIGHTS
1. What is the overall churn rate?
2. Which contract types have the highest churn rate?
3. Does monthly billing vs. non-monthly affect churn?
4. Churn rate by payment method
5. Are customers with shorter tenure more likely to churn?

In [10]:
# 1. What is the overall churn rate?

query = """
SELECT 
  COUNT(*) AS Total_Customers,
  SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned_Customers,
  ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS Churn_Rate
FROM customers;
"""

pd.read_sql_query(query, conn)

,Total_Customers,Churned_Customers,Churn_Rate
0,7032,1869,26.58


In [11]:
# 2. Which contract types have the highest churn rate?

query = """
SELECT 
  Contract,
  COUNT(*) AS Total,
  SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
  ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM customers
GROUP BY Contract
ORDER BY ChurnRate DESC
LIMIT 10;
"""

pd.read_sql_query(query, conn)


,Contract,Total,Churned,ChurnRate
0,Month-to-month,3875,1655,42.71
1,One year,1472,166,11.28
2,Two year,1685,48,2.85


In [12]:
# 3. Does monthly billing vs. non-monthly affect churn?

query = """
SELECT 
  Paperlessbilling,
  COUNT(*) AS Total,
  SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
  ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM customers
GROUP BY Paperlessbilling;
"""
pd.read_sql_query(query, conn)

,PaperlessBilling,Total,Churned,ChurnRate
0,No,2864,469,16.38
1,Yes,4168,1400,33.59


In [13]:
# 4.Churn rate by payment method

query = """
SELECT 
  paymentmethod,
  COUNT(*) AS Total,
  SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
  ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM customers
GROUP BY paymentmethod;
"""
pd.read_sql_query(query, conn)


,PaymentMethod,Total,Churned,ChurnRate
0,Bank transfer (automatic),1542,258,16.73
1,Credit card (automatic),1521,232,15.25
2,Electronic check,2365,1071,45.29
3,Mailed check,1604,308,19.20


In [14]:
# 5. Are customers with shorter tenure more likely to churn?

query = """
SELECT
  CASE 
    WHEN tenure < 12 THEN '<1 year'
    WHEN tenure BETWEEN 12 AND 24 THEN '1-2 years'
    WHEN tenure BETWEEN 25 AND 48 THEN '2-4 years'
    ELSE '4+ years'
  END AS TenureGroup,
  COUNT(*) AS Total,
  SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
  ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM customers
GROUP BY TenureGroup
ORDER BY ChurnRate DESC;
"""
pd.read_sql_query(query, conn)

,TenureGroup,Total,Churned,ChurnRate
0,<1 year,2058,999,48.54
1,1-2 years,1141,332,29.10
2,2-4 years,1594,325,20.39
3,4+ years,2239,213,9.51


SECTION B: BEHAVIOR & USAGE PATTERNS

6. Average monthly charges by internet service type 
7. Most common contract and payment method combination
8. Does having tech support reduce churn?
9. Churn by gender (to test bias or imbalance)
10. Compare churn between seniors and non-seniors

In [16]:
# 6. Average monthly charged by internet service type

query = """
SELECT
    InternetService,
    ROUND(AVG(MonthlyCharges), 2)AS AvgMonthlyCharges
FROM Customers
GROUP BY InternetService;
"""
pd.read_sql_query(query, conn)

,InternetService,AvgMonthlyCharges
0,DSL,58.09
1,Fiber optic,91.50
2,No,21.08


In [17]:
#7. Most common contract and payment method combination

query = """
SELECT Contract, PaymentMethod, COUNT (*) AS Count
FROM Customers
GROUP BY Contract, PaymentMethod
ORDER BY Count DESC
LIMIT 10;
"""
pd.read_sql_query(query, conn)

,Contract,PaymentMethod,Count
0,Month-to-month,Electronic check,1850
1,Month-to-month,Mailed check,893
2,Month-to-month,Bank transfer (automatic),589
3,Two year,Credit card (automatic),580
4,Two year,Bank transfer (automatic),562
5,Month-to-month,Credit card (automatic),543
6,One year,Credit card (automatic),398
7,One year,Bank transfer (automatic),391
8,Two year,Mailed check,375
9,One year,Electronic check,347


In [18]:
#8. Does having tech support reduce churn?

query = """
SELECT TechSupport, 
    COUNT(*) AS Total,
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
    ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM Customers
GROUP BY TechSupport
ORDER BY ChurnRate DESC;
"""

pd.read_sql_query(query, conn)


,TechSupport,Total,Churned,ChurnRate
0,No,3472,1446,41.65
1,Yes,2040,310,15.20
2,No internet service,1520,113,7.43


In [19]:
#9. Churn by gender (to test bias or imbalance)

query = """
SELECT gender, 
    COUNT(*) AS Total, 
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
    ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM Customers
GROUP BY gender;
"""
pd.read_sql_query(query, conn)

,gender,Total,Churned,ChurnRate
0,Female,3483,939,26.96
1,Male,3549,930,26.20


In [20]:
#10. Compare churn between seniors and non-seniors

query = """
SELECT SeniorCitizen, 
    COUNT(*) AS Total,
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
    ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM Customers
GROUP BY SeniorCitizen;
"""
pd.read_sql_query(query, conn)

,SeniorCitizen,Total,Churned,ChurnRate
0,0,5890,1393,23.65
1,1,1142,476,41.68


*SECTION C: PROFITABILITY & CUSTOMER VALUE*

11. Average lifetime value by churn status
12. Top 10 highest paying customers (lifetime value)
13. Do longer-tenure customers pay more monthly?
14. How many services are customers subscribed to?

In [22]:
# 11. Average lifetime value by churn status

query = """
SELECT Churn,
    ROUND(AVG(MonthlyCharges * tenure), 2) AS AverageLifetimeValue
FROM Customers
GROUP BY Churn;
"""
pd.read_sql_query(query, conn)


,Churn,AverageLifetimeValue
0,No,2555.20
1,Yes,1531.61


In [23]:
# 12. Top 10 highest paying customers (lifetime value)

query = """
SELECT customerID, tenure, MonthlyCharges,
    ROUND(AVG(MonthlyCharges * tenure), 2) AS AverageLifetimeValue
FROM Customers
GROUP BY customerID
ORDER BY AverageLifeTimeValue DESC
LIMIT 10;
"""
pd.read_sql_query(query, conn)

,customerID,tenure,MonthlyCharges,AverageLifetimeValue
0,7569-NMZYQ,72,118.75,8550.00
1,9924-JPRMC,72,118.20,8510.40
2,2889-FPWRM,72,117.80,8481.60
3,3810-DVDQQ,72,117.60,8467.20
4,9739-JLPQJ,72,117.50,8460.00
5,6904-JLBGY,72,117.35,8449.20
6,6650-BWFRT,72,117.15,8434.80
7,8984-HPEMB,71,118.65,8424.15
8,9788-HNGUT,72,116.95,8420.40
9,1488-PBLJN,72,116.85,8413.20


In [24]:
# 13. Do longer-tenure customers pay more monthly?

query = """
SELECT 
    CASE
        WHEN tenure < 12 THEN '<1 year'
        WHEN tenure BETWEEN 12 AND 24 THEN '1-2 years'
        WHEN tenure BETWEEN 24 AND 48 THEN '3-4 years'
        ELSE '4+ years'
    END as TenureGroup,
    ROUND(AVG(MonthlyCharges), 2) AS AvgMonthlyCharges
FROM Customers
GROUP BY TenureGroup;
"""
pd.read_sql_query(query, conn)


,TenureGroup,AvgMonthlyCharges
0,1-2 years,60.89
1,3-4 years,65.93
2,4+ years,73.95
3,<1 year,56.13


In [25]:
# 14. How many services are customers subscribed to?

query = """
SELECT COUNT(*) AS Customers,
       (MultipleLines != 'No phone service') +
       (OnlineSecurity != 'No internet service') +
       (OnlineBackup != 'No internet service') +
       (DeviceProtection != 'No internet service') +
       (TechSupport != 'No internet service') +
       (StreamingTV != 'No internet service') +
       (StreamingMovies != 'No internet service') AS ServiceCount
FROM customers
GROUP BY ServiceCount
ORDER BY Customers DESC;
"""
pd.read_sql_query(query, conn)

,Customers,ServiceCount
0,4832,7
1,1520,1
2,680,6


*SECTION D: DATA QUALITY & PROFILE*

15. Are there any duplicate customer IDs?
16. How many customers have missing TotalCharges or weird zero charges?
17. Distribution of tenure (how many customers per year group)

In [27]:
# 15. Are there any duplicate customer IDs?

query = """
SELECT customerID, COUNT(*)
FROM customers
GROUP BY CustomerID
HAVING COUNT(*) > 1;
"""
pd.read_sql_query(query, conn)

,customerID,COUNT(*)


In [28]:
# 16. How many customers have missing TotalCharges or weird zero charges?

query = """
SELECT COUNT(*)
FROM Customers
WHERE TotalCharges = 0 OR MonthlyCharges = 0;
"""
pd.read_sql_query(query, conn)

,COUNT(*)
0,0


In [29]:
# 17. Distribution of tenure (how many customers per year group)

query = """
SELECT tenure, COUNT(*) AS Num_Customers
FROM customers
GROUP BY tenure
ORDER BY tenure;
"""
pd.read_sql_query(query, conn)

,tenure,Num_Customers
0,1,613
1,2,238
2,3,200
3,4,176
4,5,133
...,...,...
67,68,100
68,69,95
69,70,119
70,71,170


### 1. Rank top churn-prone customers by lifetime value
### 2. Create a customer segmentation using a CTE
### 3. Find the top 3 payment methods by churn per contract type
### 4. Compare churn rate before and after 2 years tenure (pivot style)
### 5. Create a rolling average of churn by tenure (window agg)
### 6. Identify inconsistent customers (high CLTV but short tenure)
### 7. Find customers who had all services (7-in-1)
### 8. Calculate churn month-over-month (approximate using tenure)

In [31]:
# 1. Rank top churn-prone customers by lifetime value

query = """
SELECT customerID, MonthlyCharges, tenure,
       (MonthlyCharges * tenure) AS CLTV,
       RANK() OVER (ORDER BY (MonthlyCharges * tenure) DESC) AS CLTV_Rank
FROM customers
WHERE Churn = 'Yes'
LIMIT 10;
"""
pd.read_sql_query(query, conn)

,customerID,MonthlyCharges,tenure,CLTV,CLTV_Rank
0,2889-FPWRM,117.80,72,8481.60,1
1,1444-VVSGW,115.65,70,8095.50,2
2,0201-OAMXR,115.55,70,8088.50,3
3,1555-DJEQW,114.20,70,7994.00,4
4,8199-ZLLSA,118.35,67,7929.45,5
5,3886-CERTZ,109.25,72,7866.00,6
6,9053-JZFKV,116.20,67,7785.40,7
7,7317-GGVPB,108.60,71,7710.60,8
8,5271-YNWVR,113.15,68,7694.20,9
9,2834-JRTUA,108.05,71,7671.55,10


In [32]:
# 2. Create a customer segmentation using a CTE

query = """
WITH Segmented AS (
  SELECT *,
    CASE 
      WHEN tenure < 12 THEN 'New'
      WHEN tenure BETWEEN 12 AND 36 THEN 'Mid'
      ELSE 'Loyal'
    END AS Segment
  FROM customers
)
SELECT Segment, COUNT(*) AS Total,
       ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM Segmented
GROUP BY Segment;
"""
pd.read_sql_query(query, conn)


,Segment,Total,ChurnRate
0,Loyal,3001,11.93
1,Mid,1973,25.95
2,New,2058,48.54


In [33]:
# 3. Find the top 3 payment methods by churn per contract type

query = """
SELECT *
FROM (
  SELECT Contract, PaymentMethod,
         COUNT(*) AS Total,
         SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
         ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate,
         RANK() OVER (PARTITION BY Contract ORDER BY 
                      100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*) DESC) AS Rank
  FROM customers
  GROUP BY Contract, PaymentMethod
) AS Ranked
WHERE Rank <= 3;
"""
pd.read_sql_query(query, conn)

,Contract,PaymentMethod,Total,Churned,ChurnRate,Rank
0,Month-to-month,Electronic check,1850,994,53.73,1
1,Month-to-month,Bank transfer (automatic),589,201,34.13,2
2,Month-to-month,Credit card (automatic),543,178,32.78,3
3,One year,Electronic check,347,64,18.44,1
4,One year,Credit card (automatic),398,41,10.30,2
5,One year,Bank transfer (automatic),391,38,9.72,3
6,Two year,Electronic check,168,13,7.74,1
7,Two year,Bank transfer (automatic),562,19,3.38,2
8,Two year,Credit card (automatic),580,13,2.24,3


In [34]:
# 4. Compare churn rate before and after 2 years tenure (pivot style)

query = """
SELECT 
  Churn,
  SUM(CASE WHEN tenure < 24 THEN 1 ELSE 0 END) AS 'Under_2_Yrs',
  SUM(CASE WHEN tenure >= 24 THEN 1 ELSE 0 END) AS 'Over_2_Yrs'
FROM customers
GROUP BY Churn;
"""
pd.read_sql_query(query, conn)

,Churn,Under_2_Yrs,Over_2_Yrs
0,No,1797,3366
1,Yes,1308,561


In [35]:
# 5. Create a rolling average of churn by tenure (window agg)

query = """
SELECT 
  tenure,
  COUNT(*) AS Customers,
  SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
  ROUND(AVG(SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END)) OVER (
    ORDER BY tenure ROWS BETWEEN 2 PRECEDING AND CURRENT ROW), 2) AS RollingAvgChurn
FROM customers
GROUP BY tenure
ORDER BY tenure;
"""
pd.read_sql_query(query, conn)

,tenure,Customers,Churned,RollingAvgChurn
0,1,613,380,380.00
1,2,238,123,251.50
2,3,200,94,199.00
3,4,176,83,100.00
4,5,133,64,80.33
...,...,...,...,...
67,68,100,9,10.67
68,69,95,8,9.00
69,70,119,11,9.33
70,71,170,6,8.33


In [36]:
# 6. Identify inconsistent customers (high CLTV but short tenure)

query = """
SELECT customerID, MonthlyCharges, tenure,
       (MonthlyCharges * tenure) AS CLTV
FROM customers
WHERE (MonthlyCharges * tenure) > 3000 AND tenure < 6;

"""
pd.read_sql_query(query, conn)

,customerID,MonthlyCharges,tenure,CLTV


In [37]:
# 7. Find customers who had all services (7-in-1)

query = """
SELECT customerID
FROM customers
WHERE 
  MultipleLines NOT IN ('No phone service', 'No') AND
  OnlineSecurity != 'No internet service' AND
  OnlineBackup != 'No internet service' AND
  DeviceProtection != 'No internet service' AND
  TechSupport != 'No internet service' AND
  StreamingTV != 'No internet service' AND
  StreamingMovies != 'No internet service';
"""
pd.read_sql_query(query, conn)

,customerID
0,9305-CDSKC
1,1452-KIOVK
2,7892-POOKP
3,8091-TTVAX
4,0280-XJGEX
...,...
2622,6894-LFHLY
2623,0639-TSIQW
2624,6840-RESVB
2625,2234-XADUH


In [38]:
# 8. Calculate churn month-over-month (approximate using tenure)

query = """
SELECT tenure,
  COUNT(*) AS Customers,
  SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
  ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM customers
GROUP BY tenure
ORDER BY tenure;

"""
pd.read_sql_query(query, conn)

,tenure,Customers,Churned,ChurnRate
0,1,613,380,61.99
1,2,238,123,51.68
2,3,200,94,47.00
3,4,176,83,47.16
4,5,133,64,48.12
...,...,...,...,...
67,68,100,9,9.00
68,69,95,8,8.42
69,70,119,11,9.24
70,71,170,6,3.53


In [39]:
# 1. How many customers churned in each payment method?

query = """
SELECT 
    PaymentMethod, 
    COUNT(*) AS Total,
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
    ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM customers
GROUP BY PaymentMethod;
"""
pd.read_sql_query(query, conn)

,PaymentMethod,Total,Churned,ChurnRate
0,Bank transfer (automatic),1542,258,16.73
1,Credit card (automatic),1521,232,15.25
2,Electronic check,2365,1071,45.29
3,Mailed check,1604,308,19.20


In [40]:
# 2. What is the total revenue per gender?

query = """
SELECT 
    gender, 
    SUM(MonthlyCharges * tenure) AS TotalRevenue
FROM customers
GROUP BY gender;
"""
pd.read_sql_query(query, conn)

,gender,TotalRevenue
0,Female,7951414.30
1,Male,8103677.15


In [41]:
 # 3. Which internet service has the highest churn rate?

query = """
SELECT 
    InternetService, 
    COUNT(*) AS Total,
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
    ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM customers
GROUP BY InternetService;
"""
pd.read_sql_query(query, conn)

,InternetService,Total,Churned,ChurnRate
0,DSL,2416,459,19.00
1,Fiber optic,3096,1297,41.89
2,No,1520,113,7.43


In [42]:
# 4. What is the churn rate for customers with tenure < 12?

query = """
SELECT tenure,
    COUNT(*) AS Total,
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
    ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM customers
WHERE tenure < 12;
"""
pd.read_sql_query(query, conn)

,tenure,Total,Churned,ChurnRate
0,1,2058,999,48.54


In [43]:
# 5. Top 5 customers by lifetime value

query = """
SELECT customerID, 
    (MonthlyCharges * tenure) AS LifetimeValue
FROM customers
ORDER BY LifetimeValue DESC
LIMIT 5;
"""
pd.read_sql_query(query, conn)


,customerID,LifetimeValue
0,7569-NMZYQ,8550.0
1,9924-JPRMC,8510.4
2,2889-FPWRM,8481.6
3,3810-DVDQQ,8467.2
4,9739-JLPQJ,8460.0


In [44]:
#  Rank High-Value Customers with Churn Risk
## Find the top 10 customers who are spending the most (CLTV), but are currently at risk of churn.

query = """
SELECT customerID, 
        (MonthlyCharges * tenure) AS CLTV,
        MonthlyCharges, tenure
FROM Customers
WHERE Churn = 'Yes'
GROUP BY customerID
ORDER BY CLTV DESC
LIMIT 10;
"""
pd.read_sql_query(query, conn)

,customerID,CLTV,MonthlyCharges,tenure
0,2889-FPWRM,8481.60,117.80,72
1,1444-VVSGW,8095.50,115.65,70
2,0201-OAMXR,8088.50,115.55,70
3,1555-DJEQW,7994.00,114.20,70
4,8199-ZLLSA,7929.45,118.35,67
5,3886-CERTZ,7866.00,109.25,72
6,9053-JZFKV,7785.40,116.20,67
7,7317-GGVPB,7710.60,108.60,71
8,5271-YNWVR,7694.20,113.15,68
9,2834-JRTUA,7671.55,108.05,71


In [45]:
# Time-Based Churn Trend (Rolling Window)
## Create a table showing churn count and rolling average churn over 3-month tenure windows.

query = """
SELECT 
    tenure,
    COUNT(*) AS Total,
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
    ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate,
    ROUND(AVG(SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END)) OVER (
    ORDER BY tenure
    ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
 ), 2) AS RollingAvgChurn
FROM Customers
GROUP BY tenure
ORDER BY tenure;
"""
pd.read_sql_query(query, conn)


,tenure,Total,Churned,ChurnRate,RollingAvgChurn
0,1,613,380,61.99,380.00
1,2,238,123,51.68,251.50
2,3,200,94,47.00,199.00
3,4,176,83,47.16,100.00
4,5,133,64,48.12,80.33
...,...,...,...,...,...
67,68,100,9,9.00,10.67
68,69,95,8,8.42,9.00
69,70,119,11,9.24,9.33
70,71,170,6,3.53,8.33


In [46]:
# Most Common Service Bundles Among Loyal Customers
## What are the most frequent combinations of services for customers who stayed longer than 24 months?

query = """
SELECT 
  CONCAT_WS('|', 
    MultipleLines, 
    OnlineSecurity, 
    OnlineBackup, 
    DeviceProtection, 
    TechSupport, 
    StreamingTV, 
    StreamingMovies
  ) AS ServiceBundle,
  COUNT(*) AS Count
FROM customers
WHERE tenure > 24
GROUP BY ServiceBundle
ORDER BY Count DESC
LIMIT 10;
"""
pd.read_sql_query(query, conn)

,ServiceBundle,Count
0,No|No internet service|No internet service|No ...,512
1,Yes|No internet service|No internet service|No...,262
2,Yes|Yes|Yes|Yes|Yes|Yes|Yes,206
3,Yes|No|Yes|Yes|No|Yes|Yes,122
4,Yes|No|Yes|Yes|Yes|Yes|Yes,110
5,Yes|No|No|Yes|No|Yes|Yes,82
6,Yes|Yes|No|Yes|Yes|Yes|Yes,65
7,Yes|Yes|Yes|Yes|No|Yes|Yes,63
8,Yes|No|No|Yes|Yes|Yes|Yes,58
9,Yes|No|Yes|No|No|Yes|Yes,55


In [47]:
# Churn Probability Segments via CTE
## Segment customers into: 
 #Low Risk if tenure > 24 AND no PaperlessBilling
 #Medium Risk if tenure between 12–24 OR has OnlineSecurity
 #High Risk if tenure < 12 AND has PaperlessBilling. Then get churn rate per segment.

query = """
WITH RiskSegmented AS (
    SELECT customerID, Churn,
     CASE 
         WHEN tenure > 24 AND PaperlessBilling = 'No' THEN 'Low_Risk'
         WHEN tenure BETWEEN 12 AND 24 OR OnlineSecurity = 'Yes' THEN 'Medium_Risk'
         WHEN tenure < 12 AND PaperlessBilling = 'Yes' THEN 'High_Risk'
        ELSE 'Unclassified'
    END AS RiskSegment
FROM Customers
)

SELECT RiskSegment,
    COUNT(*) AS Total,
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
    ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM RiskSegmented
GROUP BY RiskSegment
ORDER BY ChurnRate DESC;
"""
pd.read_sql_query(query, conn)

,RiskSegment,Total,Churned,ChurnRate
0,High_Risk,1079,655,60.70
1,Unclassified,2098,585,27.88
2,Medium_Risk,2296,515,22.43
3,Low_Risk,1559,114,7.31


In [48]:
query = """
SELECT customerID,
    COUNT(*) Total,
    CASE WHEN tenure > 24 AND PaperlessBilling = 'No' THEN 'Low_Risk'
         WHEN tenure BETWEEN 12 AND 24 OR OnlineSecurity = 'Yes' THEN 'Medium_Risk'
         WHEN tenure < 12 AND PaperlessBilling = 'Yes' THEN 'High_Risk'
        ELSE 'No_Risk'
    END AS TenureRisk,
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
    ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM Customers
GROUP BY TenureRisk
ORDER BY ChurnRate;
"""
pd.read_sql_query(query, conn)

,customerID,Total,TenureRisk,Churned,ChurnRate
0,5575-GNVDE,1559,Low_Risk,114,7.31
1,3668-QPYBK,2296,Medium_Risk,515,22.43
2,7892-POOKP,2098,No_Risk,585,27.88
3,7590-VHVEG,1079,High_Risk,655,60.70


In [49]:
# Which Combinations Drive the Most Churn?
## Which contract + payment method combinations result in the highest churn?

query = """
SELECT Contract, PaymentMethod,
    COUNT(*) AS Total,
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned, 
    ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM Customers
GROUP BY Contract, PaymentMethod
ORDER BY ChurnRate DESC;
"""
pd.read_sql_query(query, conn)

,Contract,PaymentMethod,Total,Churned,ChurnRate
0,Month-to-month,Electronic check,1850,994,53.73
1,Month-to-month,Bank transfer (automatic),589,201,34.13
2,Month-to-month,Credit card (automatic),543,178,32.78
3,Month-to-month,Mailed check,893,282,31.58
4,One year,Electronic check,347,64,18.44
5,One year,Credit card (automatic),398,41,10.30
6,One year,Bank transfer (automatic),391,38,9.72
7,Two year,Electronic check,168,13,7.74
8,One year,Mailed check,336,23,6.85
9,Two year,Bank transfer (automatic),562,19,3.38


In [50]:
query = """
SELECT *
FROM (
  SELECT Contract, PaymentMethod,
         COUNT(*) AS Total,
         SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
         ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate,
         RANK() OVER (PARTITION BY Contract ORDER BY 
                      SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*) DESC) AS Rank
  FROM customers
  GROUP BY Contract, PaymentMethod
) AS ranked
WHERE Rank <= 3;
"""
pd.read_sql_query(query, conn)

,Contract,PaymentMethod,Total,Churned,ChurnRate,Rank
0,Month-to-month,Mailed check,893,282,31.58,1
1,Month-to-month,Electronic check,1850,994,53.73,1
2,Month-to-month,Credit card (automatic),543,178,32.78,1
3,Month-to-month,Bank transfer (automatic),589,201,34.13,1
4,One year,Mailed check,336,23,6.85,1
5,One year,Electronic check,347,64,18.44,1
6,One year,Credit card (automatic),398,41,10.30,1
7,One year,Bank transfer (automatic),391,38,9.72,1
8,Two year,Mailed check,375,3,0.80,1
9,Two year,Electronic check,168,13,7.74,1


In [108]:
import pandas as pd
import sqlite3

# Connect to your database
conn = sqlite3.connect('telco_data.db')  # make sure this DB has 'customers' table loaded

# 1. What is the overall churn rate?
query1 = """
SELECT COUNT(*) AS Total, 
       SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
       ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM customers;
"""

# 2. Highest churn rate by contract type
query2 = """
SELECT 
  Contract,
  COUNT(*) AS Total,
  SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
  ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM customers
GROUP BY Contract
ORDER BY ChurnRate DESC
LIMIT 10;
"""

# 3. Churn Rate by payment method
query3 = """
SELECT 
  PaymentMethod,
  COUNT(*) AS Total,
  SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
  ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM customers
GROUP BY PaymentMethod;
"""

# 4. Top 10 churned customers by CLTV
query4 = """
SELECT customerID, 
        (MonthlyCharges * tenure) AS CLTV,
        MonthlyCharges, tenure
FROM customers
WHERE Churn = 'Yes'
ORDER BY CLTV DESC
LIMIT 10;
"""

# 5. ServiceBundles
query5 = """
SELECT 
  MultipleLines || '|' || OnlineSecurity || '|' || OnlineBackup || '|' || 
  DeviceProtection || '|' || TechSupport || '|' || StreamingTV || '|' || StreamingMovies AS ServiceBundle,
  COUNT(*) AS Count
FROM customers
WHERE tenure > 24
GROUP BY ServiceBundle
ORDER BY Count DESC
LIMIT 10;
"""

# 6. RiskSegmentation
query6 = """
SELECT
    CASE 
        WHEN tenure > 24 AND PaperlessBilling = 'No' THEN 'Low_Risk'
        WHEN tenure BETWEEN 12 AND 24 OR OnlineSecurity = 'Yes' THEN 'Medium_Risk'
        WHEN tenure < 12 AND PaperlessBilling = 'Yes' THEN 'High_Risk'
        ELSE 'No_Risk'
    END AS TenureRisk,
    COUNT(*) AS Total,
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
    ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM customers
GROUP BY TenureRisk
ORDER BY ChurnRate DESC;
"""

# 7. Gender-based churn
query7 = """
SELECT gender, 
    COUNT(*) AS Total, 
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
    ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM customers
GROUP BY gender;
"""

# Save all to same Excel file with different sheets
with pd.ExcelWriter("telco_churn_dashboard_data.xlsx") as writer:
    pd.read_sql_query(query1, conn).to_excel(writer, sheet_name="ChurnSummary", index=False)
    pd.read_sql_query(query2, conn).to_excel(writer, sheet_name="ChurnByContract", index=False)
    pd.read_sql_query(query3, conn).to_excel(writer, sheet_name="ChurnByPayment", index=False)
    pd.read_sql_query(query4, conn).to_excel(writer, sheet_name="TopCLTVChurn", index=False)
    pd.read_sql_query(query5, conn).to_excel(writer, sheet_name="ServiceBundles", index=False)
    pd.read_sql_query(query6, conn).to_excel(writer, sheet_name="RiskSegment", index=False)
    pd.read_sql_query(query7, conn).to_excel(writer, sheet_name="ChurnByGender", index=False)

print("Excel file with multiple sheets saved successfully!")import pandas as pd
import sqlite3

# Connect to your database
conn = sqlite3.connect('telco_data.db')  # make sure this DB has 'customers' table loaded

# 1. What is the overall churn rate?
query1 = """
SELECT COUNT(*) AS Total, 
       SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
       ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM customers;
"""

# 2. Highest churn rate by contract type
query2 = """
SELECT 
  Contract,
  COUNT(*) AS Total,
  SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
  ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM customers
GROUP BY Contract
ORDER BY ChurnRate DESC
LIMIT 10;
"""

# 3. Churn Rate by payment method
query3 = """
SELECT 
  PaymentMethod,
  COUNT(*) AS Total,
  SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
  ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM customers
GROUP BY PaymentMethod;
"""

# 4. Top 10 churned customers by CLTV
query4 = """
SELECT customerID, 
        (MonthlyCharges * tenure) AS CLTV,
        MonthlyCharges, tenure
FROM customers
WHERE Churn = 'Yes'
ORDER BY CLTV DESC
LIMIT 10;
"""

# 5. ServiceBundles
query5 = """
SELECT 
  MultipleLines || '|' || OnlineSecurity || '|' || OnlineBackup || '|' || 
  DeviceProtection || '|' || TechSupport || '|' || StreamingTV || '|' || StreamingMovies AS ServiceBundle,
  COUNT(*) AS Count
FROM customers
WHERE tenure > 24
GROUP BY ServiceBundle
ORDER BY Count DESC
LIMIT 10;
"""

# 6. RiskSegmentation
query6 = """
SELECT
    CASE 
        WHEN tenure > 24 AND PaperlessBilling = 'No' THEN 'Low_Risk'
        WHEN tenure BETWEEN 12 AND 24 OR OnlineSecurity = 'Yes' THEN 'Medium_Risk'
        WHEN tenure < 12 AND PaperlessBilling = 'Yes' THEN 'High_Risk'
        ELSE 'No_Risk'
    END AS TenureRisk,
    COUNT(*) AS Total,
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
    ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM customers
GROUP BY TenureRisk
ORDER BY ChurnRate DESC;
"""

# 7. Gender-based churn
query7 = """
SELECT gender, 
    COUNT(*) AS Total, 
    SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
    ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
FROM customers
GROUP BY gender;
"""

with pd.ExcelWriter("telco_churn_dashboard_data.xlsx") as writer:
    pd.read_sql_query(query1, conn).to_excel(writer, sheet_name="ChurnSummary", index=False)
    pd.read_sql_query(query2, conn).to_excel(writer, sheet_name="ChurnByContract", index=False)
    pd.read_sql_query(query3, conn).to_excel(writer, sheet_name="ChurnByPayment", index=False)
    pd.read_sql_query(query4, conn).to_excel(writer, sheet_name="TopCLTVChurn", index=False)
    pd.read_sql_query(query5, conn).to_excel(writer, sheet_name="ServiceBundles", index=False)
    pd.read_sql_query(query6, conn).to_excel(writer, sheet_name="RiskSegment", index=False)
    pd.read_sql_query(query7, conn).to_excel(writer, sheet_name="ChurnByGender", index=False)

print("Excel file saved!")

SyntaxError: invalid syntax (82628314.py, line 100)

In [110]:
import pandas as pd
import sqlite3

# Connect to your database
conn = sqlite3.connect('telco_data.db')

# Define your queries with labels
queries = {
    "ChurnSummary": """
        SELECT COUNT(*) AS Total, 
               SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
               ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
        FROM customers;
    """,
    "ChurnByContract": """
        SELECT Contract,
               COUNT(*) AS Total,
               SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
               ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
        FROM customers
        GROUP BY Contract
        ORDER BY ChurnRate DESC;
    """,
    "ChurnByPayment": """
        SELECT PaymentMethod,
               COUNT(*) AS Total,
               SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
               ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
        FROM customers
        GROUP BY PaymentMethod;
    """,
    "TopCLTVChurn": """
        SELECT customerID, 
               (MonthlyCharges * tenure) AS CLTV,
               MonthlyCharges, tenure
        FROM customers
        WHERE Churn = 'Yes'
        ORDER BY CLTV DESC
        LIMIT 10;
    """,
    "ServiceBundles": """
        SELECT 
          MultipleLines || '|' || OnlineSecurity || '|' || OnlineBackup || '|' || 
          DeviceProtection || '|' || TechSupport || '|' || StreamingTV || '|' || StreamingMovies AS ServiceBundle,
          COUNT(*) AS Count
        FROM customers
        WHERE tenure > 24
        GROUP BY ServiceBundle
        ORDER BY Count DESC
        LIMIT 10;
    """,
    "RiskSegment": """
        SELECT
            CASE 
                WHEN tenure > 24 AND PaperlessBilling = 'No' THEN 'Low_Risk'
                WHEN tenure BETWEEN 12 AND 24 OR OnlineSecurity = 'Yes' THEN 'Medium_Risk'
                WHEN tenure < 12 AND PaperlessBilling = 'Yes' THEN 'High_Risk'
                ELSE 'No_Risk'
            END AS TenureRisk,
            COUNT(*) AS Total,
            SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
            ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
        FROM customers
        GROUP BY TenureRisk
        ORDER BY ChurnRate DESC;
    """,
    "ChurnByGender": """
        SELECT gender, 
               COUNT(*) AS Total, 
               SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
               ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
        FROM customers
        GROUP BY gender;
    """
}

# Combine all into one big DataFrame with a source column
combined_df = pd.DataFrame()

for label, sql in queries.items():
    try:
        df = pd.read_sql_query(sql, conn)
        df.insert(0, "Source", label)  # Add label column
        combined_df = pd.concat([combined_df, df], ignore_index=True)
        print(f" Loaded: {label}")
    except Exception as e:
        print(f" Failed: {label} — {e}")

# Save to one CSV file
combined_df.to_csv("telco_churn_dashboard_data.csv", index=False)
print("Saved: telco_churn_dashboard_data.csv")


 Failed: ChurnSummary — Execution failed on sql '
        SELECT COUNT(*) AS Total, 
               SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
               ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
        FROM customers;
    ': no such table: customers
 Failed: ChurnByContract — Execution failed on sql '
        SELECT Contract,
               COUNT(*) AS Total,
               SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
               ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*), 2) AS ChurnRate
        FROM customers
        GROUP BY Contract
        ORDER BY ChurnRate DESC;
    ': no such table: customers
 Failed: ChurnByPayment — Execution failed on sql '
        SELECT PaymentMethod,
               COUNT(*) AS Total,
               SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) AS Churned,
               ROUND(100.0 * SUM(CASE WHEN Churn = 'Yes' THEN 1 ELSE 0 END) / COUNT(*),